# Credit Risk Resampling Techniques

In [82]:
import warnings
warnings.filterwarnings('ignore')

In [83]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Read the CSV into DataFrame

In [85]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [86]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status']

In [87]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [88]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk     75036
high_risk     2500
Name: count, dtype: int64

In [90]:
# Convert categorial fields to numerics and change labels to 0 and 1
X_dummies = pd.get_dummies(X)
y_label = LabelEncoder().fit_transform(y)

In [91]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_dummies, y_label, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [92]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [93]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaled_train = scaler.fit(X_train)

In [94]:
# Scale the training and testing data
X_train_scaled = scaled_train.transform(X_train)
X_test_scaled = scaled_train.transform(X_test)

# Simple Logistic Regression

In [95]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [97]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9889115309798473

In [98]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  609,    10],
       [  113, 18652]])

In [100]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      0.98      0.99      0.91      0.99      0.98       619
          1       1.00      0.99      0.98      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [101]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({1: 56271, 0: 56271})

In [102]:
# Train the Logistic Regression model using the resampled data
model_ros = LogisticRegression(solver='lbfgs', random_state=1)
model_ros.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [103]:
# Calculated the balanced accuracy score
y_pred_ros = model_ros.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_ros)

0.9934649587814939

In [104]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_ros)

array([[  615,     4],
       [  124, 18641]])

In [105]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_ros))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.83      0.99      0.99      0.91      0.99      0.99       619
          1       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [106]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled_sm, y_resampled_sm = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)

# View the count of target classes with Counter
Counter(y_resampled_sm)

Counter({1: 56271, 0: 56271})

In [107]:
# Train the Logistic Regression model using the resampled data
model_sm = LogisticRegression(solver='lbfgs', random_state=1)
model_sm.fit(X_resampled_sm, y_resampled_sm)

LogisticRegression(random_state=1)

In [108]:
# Calculated the balanced accuracy score
y_pred_sm = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_sm)

0.9889115309798473

In [109]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_sm)

array([[  609,    10],
       [  113, 18652]])

In [110]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_sm))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      0.98      0.99      0.91      0.99      0.98       619
          1       1.00      0.99      0.98      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [111]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids()
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_cc)

Counter({0: 1881, 1: 1881})

In [112]:
# Train the Logistic Regression model using the resampled data
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled_cc, y_resampled_cc)

LogisticRegression(random_state=1)

In [113]:
# Calculate the balanced accuracy score
y_pred_cc = model_cc.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_cc)

0.9929503031930944

In [114]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_cc)

array([[  614,     5],
       [  113, 18652]])

In [115]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      0.99      0.99      0.91      0.99      0.99       619
          1       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [116]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smoteen = SMOTEENN(random_state=1)
X_resampled_teen, y_resampled_teen = smoteen.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_teen)

Counter({1: 55937, 0: 55509})

In [117]:
# Train the Logistic Regression model using the resampled data
model_teen = LogisticRegression(solver='lbfgs', random_state=1)
model_teen.fit(X_resampled_teen, y_resampled_teen)

LogisticRegression(random_state=1)

In [118]:
# Calculate the balanced accuracy score
y_pred_teen = model_teen.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_teen)

0.9934649587814939

In [119]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_teen)

array([[  615,     4],
       [  124, 18641]])

In [120]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_teen))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.83      0.99      0.99      0.91      0.99      0.99       619
          1       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   Both the Naive Random Oversampling and SMOTEEN models had similar accuracy scores of 99.35%

2. Which model had the best recall score?

    There wasn't a clear winner in regards to recall, as the Cluster Centroids, SMOTEEN, and Naive Random Oversampling had recall scores of 0.99

3. Which model had the best geometric mean score?

    All of the models had a geometric mean scsore of 0.99, so there wasn't a clear winner.
